<img src="../common/rfsoc_book_banner.jpg" alt="University of Strathclyde" align="left">

<div class="alert alert-block" style="background-color: #c7b8d6; padding: 10px">
    <p style="color: #222222">
        <b>Note:</b>
        <br>
        This Jupyter notebook uses hardware features of the Zynq UltraScale+ RFSoC device. Therefore, the notebook cells will only execute successfully on an RFSoC platform.
    </p>
</div>

# Notebook Set I

---

## 03 - RFSoC OFDM Transceiver
This notebook demonstrates the implementation of an Orthogonal Frequency Division Multiplexing (OFDM) transceiver on RFSoC. PYNQ is used to control the underlying modulation scheme of the OFDM sub-carriers and for visualisation of data at various stages in the transmit/receive chain, such as the received constellations.

## Table of Contents
* [1. Introduction](#introduction)
    * [1.1 Hardware Setup](#hardware-setup)
    * [1.2 Software Setup](#software-setup)
* [2. OFDM Transmitter](#ofdm-transmitter)
* [3. OFDM Receiver](#ofdm-receiver)
* [4. Conclusion](#conclusion)

## Revision
* **v1.0** | 05/01/23 | *First Revision*
* **v1.1** | 19/05/23 | *Fix header numbering*

---


## 1. Introduction <a class="anchor" id="introduction"></a>
The demonstrator is a complete OFDM transceiver. This notebook will explain each stage of the system with a combination of text, diagrams and live data capture. Figure 1 below provides an overview of the system.

<figure>
<img src="./images/ofdm_architecture.png" height='75%' width='75%'/>
    <figcaption><b>Figure 1: OFDM demonstrator system overview.</b></figcaption>
</figure>

The OFDM system starts with generation of random data symbols from 1 of 10 possible modulation schemes (BPSK to 1024 QAM), based on input provided from PYNQ. In accordance with the procedure used in the IEEE 802.11a/g standard, the symbols are grouped into blocks of 48 for mapping to sub-carriers. The OFDM symbol consists of 48 data sub-carriers, 4 pilot sub-carriers and 12 null sub-carriers (including DC). The final OFDM symbol is created by performing a 64 point IFFT and adding a 16 sample Cyclic Prefix (CP). The transmitted signal consists of a continuous stream of OFDM symbols. At the very beginning of the data stream, the L-STF and L-LTF training symbols from the IEEE 802.11a/g standard are transmitted, to aid synchronisation and channel estimation tasks in the receiver. 

In the receiver, timing and frequency synchronisation are performed to acquire symbol timing and correct for any frequency offsets. Once timing is achieved, the FFT is performed to recover the underlying data symbols in each OFDM symbol. The L-LTF symbols are used to estimate the channel frequency response at each sub-carrier position, which subsequently allows the data to be equalised. Finally, the pilot symbols are used to correct for residual phase errors in the phase tracking stage. The recovered symbols are then passed into the PS for visualisation in PYNQ.

### 1.1. Hardware Setup <a class="anchor" id="hardware-setup"></a>
Your ZCU111 development board should be setup in single channel mode. There are several SMA interfaces on your development board. To setup your board for this demonstration, you can connect a loopback channel as shown in Figure 2.

The default loopback configuration in this demonstration is connected as follows:
* Channel 0: DAC6 (Tile 229 Block 2) to ADC 0 (Tile 224 Block 0)

<figure>
<img src='images/zcu111_setup.png' height='50%' width='50%'/>
    <figcaption><b>Figure 2: ZCU111 development board setup in loopback mode.</b></figcaption>
</figure>

The loopback connection will be useful for running the OFDM demonstrator. **Do Not** attach an antenna to any SMA interfaces labelled DAC.

<div class="alert alert-box alert-danger">
<b>Caution:</b>
    In this demonstration, we generate signals using the RFSoC development board. Your device should be setup in loopback mode. You should understand that the RFSoC platform can also transmit RF signals wirelessly. Remember that unlicensed wireless transmission of RF signals may be illegal in your geographical location. Radio signals may also interfere with nearby devices, such as pacemakers and emergency radio equipment. Note that it is also illegal to intercept and decode particular RF signals. If you are unsure, please seek professional support.
</div>

### 1.2. Software Setup
The setup for the OFDM demonstration system is nearly complete. The majority of the libraries used by the demonstrator design are contained inside the RFSoC-OFDM software package. We only need to run a few code cells to initialise the environment.

In [ ]:
from rfsoc_ofdm.overlay import Overlay
import ipywidgets as ipw

ofdm_hw = Overlay()

## 2. OFDM Transmitter <a class="anchor" id="ofdm-transmitter"></a>
There are a total of 10 modulation schemes available to be transmitted. These are generated on the programmable logic and can be chosen between by updating the *ofdm_tx IP core's* **mod** register with a value from 0-9 over AXI4-Lite. Figure 3 illustrates the IP core as a simplified block diagram.

<figure>
<img src="./images/ofdm_modulation_selector.png" height='35%' width='35%'/>
    <figcaption><b>Figure 3: Symbol generation block diagram.</b></figcaption>
</figure>

This drop down widget sends the value associated with each modulation scheme to the ofdm_tx core. Run the cell below to use it.

In [ ]:
ofdm_hw.ofdm_transmitter.modulation_dropdown.get_widget()

The output of the symbol generation block has been tapped off, allowing for the live symbols to be visualised in Jupyter Lab. Run the cell below and hit play on the chart to inspect the symbols generated on the programmable logic.

In [ ]:
ipw.VBox([ofdm_hw.inspectors['transmitter'].time_plot(),
          ofdm_hw.inspectors['transmitter'].plot_control()])

## 3. OFDM Receiver
Run the cell below to see the output of the OFDM receiver displayed as a constellation.

In [ ]:
ipw.VBox([ofdm_hw.inspectors['constellation'].constellation_plot(),
          ofdm_hw.inspectors['constellation'].plot_control()])

Try changing the modulation scheme using the code cell that was ran earlier. You should be able to visualise the modulation schemes in the plot above.

## 4. Conclusion
This notebook has demonstrated a live OFDM transceiver operating on an RFSoC. It has been shown how PYNQ can be used to interact with various parts of the hardware design, offering control of the system and visualisation of data. 

* The various components comprising the OFDM transmitter and receiver have been introduced at a high level. 
    * Modulation symbols were inspected.
    * The received and sychronised constellation were plotted.
* Interacted with a real-time RF system.
    * Changed modulation scheme.
    
Congratulations! You have completed all practical exercises in the RFSoC Book.

---

[⬅️ Previous Notebook](02_ofdm_python_transceiver.ipynb)

Copyright © 2023 Strathclyde Academic Media

---
---